# 3. Using the Convolution Operation in Neural Networks

### About this notebook

This notebook was used in the 50.039 Deep Learning course at the Singapore University of Technology and Design.

**Author:** Matthieu DE MARI (matthieu_demari@sutd.edu.sg)

**Version:** 1.0 (27/12/2022)

**Requirements:**
- Python 3 (tested on v3.9.6)
- Matplotlib (tested on v3.5.1)
- Numpy (tested on v1.22.1)
- Pillow (tested on v9.3.0)

### Imports and CUDA

In [1]:
# Matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
# Numpy
import numpy as np
from numpy.random import default_rng
# OS
import os
# Pillow
from PIL import Image
# Time
from time import time
# Torch
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize
from torchvision.datasets import MNIST
import torch.nn.functional as F
import torch.nn as nn

In [2]:
# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### MNIST Dataset

...

In [3]:
# Define transform to convert images to tensors and normalize them
transform_data = Compose([ToTensor(),
                          Normalize((0.1307,), (0.3081,))])

# Load the data
batch_size = 256
train_dataset = MNIST(root='./mnist/', train = True, download = True, transform = transform_data)
test_dataset = MNIST(root='./mnist/', train = False, download = True, transform = transform_data)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

### Using Conv2d in our model

...

In [4]:
class MNIST_CNN(nn.Module):
    def __init__(self):
        super(MNIST_CNN, self).__init__()
        
        # Two convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1)

        # Two fully connected layers
        self.fc1 = nn.Linear(64*28*28, 128) # 64*28*28 = 50176
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Display initial shape
        print("Initial: ", x.shape)
        
        # Pass input through first convolutional layer
        x = self.conv1(x)
        x = F.relu(x)
        print("After conv1: ", x.shape)

        # Pass output of first conv layer through second convolutional layer
        x = self.conv2(x)
        x = F.relu(x)
        print("After conv2: ", x.shape)

        # Flatten output of second conv layer
        x = x.view(-1, 64*28*28)
        print("After flatten: ", x.shape)

        # Pass flattened output through first fully connected layer
        x = self.fc1(x)
        x = F.relu(x)
        print("After FC1: ", x.shape)

        # Pass output of first fully connected layer through second fully connected layer
        x = self.fc2(x)
        print("After FC2: ", x.shape)
        return x

In [5]:
model = MNIST_CNN()
print(model.modules)

<bound method Module.modules of MNIST_CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=50176, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)>


In [6]:
for inputs, labels in train_loader:
    out = model(inputs)
    break

Initial:  torch.Size([256, 1, 28, 28])
After conv1:  torch.Size([256, 32, 28, 28])
After conv2:  torch.Size([256, 64, 28, 28])
After flatten:  torch.Size([256, 50176])
After FC1:  torch.Size([256, 128])
After FC2:  torch.Size([256, 10])


### What's next?

...